In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [6]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [7]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [8]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [9]:
def getStakesAndAprs():
    stakes = dict()
    aprs = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            effective_stake = 0
            for j in i['metrics']['by-bls-key']:
                effective_stake += float(j['key']['effective-stake'])
       
            apr = float(i['lifetime']['apr'])
            stakes[address] = effective_stake
            aprs[address] = apr
    return stakes, aprs

In [10]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [11]:
def R11_test(single):
    logger.info(f"Test-R11: Earning is proportional to effective stake ")
    num = 1
    iterations = 0
    if single:
        curr_test = None
    else:
        curr_test = R14_test
    
    while iterations < num:
        logger.info(f"test {iterations+1} will begin ...")
        block, last_block = getCurrentAndLastBlock()
        logger.info(f"current and last block numbers: {block}, {last_block}")
        if block == last_block:
            new_block = block+1
            while block < new_block:
                block = getBlockNumber()
            block, last_block = getCurrentAndLastBlock()
            logger.info(f"current and last block numbers: {block}, {last_block}")
        epoch = getEpoch()

        second_last_block = last_block - 1
        while block < second_last_block:
            block = getBlockNumber()
        logger.info(f"second last block in current epoch reached {block}, will wait for 6 seconds...")
        time.sleep(6)
        logger.info(f"begin collecting stakes and aprs...")
        stakes, aprs = getStakesAndAprs()

        # in the last block, we can not get the total effective stakes, no metrics. 
        new_block = block + 2
        while block < new_block:
            block = getBlockNumber()
        logger.info(f"first block in new epoch reached, {block}, will wait for 5 seconds...")
        time.sleep(5)
        logger.info(f"compare the changes...")
        new_stakes, new_aprs = getStakesAndAprs()

        apr_diff = diffAndFilter(aprs, new_aprs)
        # get the validators whose effective stake changes
        stake_diff = diffAndFilter(stakes, new_stakes)

        if not stake_diff:
            logger.info(f"in this iteration, no validators change the effective stake\n")
            return "Need More Tests", curr_test
        if not apr_diff:
            logger.info(f"in this iteration, no validators change the apr\n")
            return "Need More Tests", curr_test

        flag = True
        for k,v in stake_diff.items():
            if k in apr_diff:
                if v > 0: 
                    if apr_diff[k] <= 0:
                        flag = False
                        logger.warning(f"Test-R11: Fail")
                        logger.warning(f"{k}'s effective stake increase: {v}")
                        logger.warning(f"but apr doesn't increase, apr changes: {apr_diff[k]}\n")
                if v < 0:
                    if apr_diff[k] >= 0:
                        flag = False
                        logger.warning(f"Test-R11: Fail")
                        logger.warning(f"{k}'s effective stake decrease: {v}")
                        logger.warning(f"apr doesn't decrease, apr changes: {apr_diff[k]}\n")
        iterations += 1 

    if flag:
        logger.info(f"Test-R11: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [12]:
R11_test(True)

INFO:economic-test:Test-R11: Earning is proportional to effective stake 
INFO:economic-test:test 1 will begin ...
INFO:economic-test:current and last block numbers: 129139, 129161


current and last block numbers 129139 129161


INFO:economic-test:second last block in current epoch reached 129160, will wait for 6 seconds...
INFO:economic-test:begin collecting stakes and aprs...
INFO:economic-test:first block in new epoch reached, 129162, will wait for 5 seconds...
INFO:economic-test:compare the changes...


















(False, None)